In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df_credits = pd.read_csv('../data/raw/tmdb_5000_credits.csv')
df_movies = pd.read_csv('../data/raw/tmdb_5000_movies.csv')

In [3]:
df_credits.sample(1)

,movie_id,title,cast,crew
3440,338,"Good bye, Lenin!","[{""cast_id"": 4, ""character"": ""Alexander 'Alex'...","[{""credit_id"": ""52fe4239c3a36847f800d9c5"", ""de..."


In [4]:
df_movies.sample(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4513,0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",http://www.benji.com/MeetBenji.htm,23069,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 5608,...",en,Benji,A stray dog saves two kidnapped children.,2.651418,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1974-10-17,0,86.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Benji,6.1,35


In [5]:
df = df_credits.merge(df_movies, on ='title')

In [ ]:
df.columns

Index(['movie_id', 'title', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'id', 'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'vote_average', 'vote_count'],
      dtype='object')

In [6]:
new_df = df.drop(columns = ['budget', 'homepage', 'movie_id', 'original_language', 'popularity', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'vote_average', 'vote_count', 'tagline', 'production_companies'])

In [7]:
new_df.sample(1)

,title,cast,crew,genres,id,keywords,original_title,overview
1334,Magnolia,"[{""cast_id"": 3, ""character"": ""Linda Partridge""...","[{""credit_id"": ""52fe4239c3a36847f800d6cb"", ""de...","[{""id"": 18, ""name"": ""Drama""}]",334,"[{""id"": 455, ""name"": ""farewell""}, {""id"": 802, ...",Magnolia,An epic mosaic of many interrelated characters...


In [8]:
import ast

In [9]:
# we have to find only top 3 casts:
def find_cast(cast):
    casts = []
    count = 0
    for i in ast.literal_eval(cast):
        if count!=3:
            casts.append(i['character'])
            count+=1
        else: break
        
    return casts

In [10]:
new_df['cast'] = new_df['cast'].apply(find_cast)

In [11]:
def find_director(object):
    director = []
    for i in ast.literal_eval(object):
        if i['job'] == 'Director':
            director.append(i['name'])
    return director

In [12]:
new_df['crew'] = new_df['crew'].apply(find_director)

In [13]:
new_df['genres'][1]

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

In [14]:
def find_genre(object):
    genre = []
    for i in ast.literal_eval(object):
        genre.append(i['name'])
    return genre

In [15]:
new_df['genres'] = new_df['genres'].apply(find_genre)

In [16]:
new_df['keywords'] = new_df['keywords'].apply(find_genre)

In [17]:
new_df = new_df.dropna()

In [18]:
new_df['overview'] = new_df['overview'].apply(lambda x: x.split())

In [19]:
new_df = new_df.drop(columns = 'original_title')

In [20]:
def collapse_names(lst):
    return [name.replace(" ", "") for name in lst]

cols = ['cast', 'crew', 'genres', 'keywords']

for col in cols:
    new_df[col] = new_df[col].apply(collapse_names)


In [21]:
new_df

,title,cast,crew,genres,id,keywords,overview
0,Avatar,"[JakeSully, Neytiri, Dr.GraceAugustine]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,Pirates of the Caribbean: At World's End,"[CaptainJackSparrow, WillTurner, ElizabethSwann]",[GoreVerbinski],"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d..."
2,Spectre,"[JamesBond, Blofeld, Madeleine]",[SamMendes],"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send..."
3,The Dark Knight Rises,"[BruceWayne/Batman, AlfredPennyworth, JamesGor...",[ChristopherNolan],"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney..."
4,John Carter,"[JohnCarter, DejahThoris, Sola]",[AndrewStanton],"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...,...,...,...,...
4804,El Mariachi,"[ElMariachi, Bigotón, Mauricio(Moco)]",[RobertRodriguez],"[Action, Crime, Thriller]",9367,"[unitedstates–mexicobarrier, legs, arms, paper...","[El, Mariachi, just, wants, to, play, his, gui..."
4805,Newlyweds,"[Buzzy, Linda, Marsha]",[EdwardBurns],"[Comedy, Romance]",72766,[],"[A, newlywed, couple's, honeymoon, is, upended..."
4806,"Signed, Sealed, Delivered","[OliverO’Toole, ShaneMcInerney, RitaHaywith]",[ScottSmith],"[Comedy, Drama, Romance, TVMovie]",231617,"[date, loveatfirstsight, narration, investigat...","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,Shanghai Calling,"[Sam, Amanda, Donald]",[DanielHsia],[],126186,[],"[When, ambitious, New, York, attorney, Sam, is..."


In [22]:
new_df['tags'] = new_df['genres'] + new_df['cast'] + new_df['overview'] + new_df['keywords'] + new_df['crew']

In [23]:
new_df = new_df.drop(columns = ['cast', 'crew', 'overview', 'keywords', 'genres'])

In [24]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [25]:
movies = new_df

In [26]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [27]:
movies

,title,id,tags
0,Avatar,19995,action adventure fantasy sciencefiction jakesu...
1,Pirates of the Caribbean: At World's End,285,adventure fantasy action captainjacksparrow wi...
2,Spectre,206647,action adventure crime jamesbond blofeld madel...
3,The Dark Knight Rises,49026,action crime drama thriller brucewayne/batman ...
4,John Carter,49529,action adventure sciencefiction johncarter dej...
...,...,...,...
4804,El Mariachi,9367,action crime thriller elmariachi bigotón mauri...
4805,Newlyweds,72766,comedy romance buzzy linda marsha a newlywed c...
4806,"Signed, Sealed, Delivered",231617,comedy drama romance tvmovie olivero’toole sha...
4807,Shanghai Calling,126186,sam amanda donald when ambitious new york atto...


**Remove Punctuation**

In [28]:
import re
def remove_punctuation(text):

    text = re.sub(r"[^a-z\s]", " ", text)
    return text


In [29]:
movies['tags'] = movies['tags'].apply(remove_punctuation)

In [30]:
movies

,title,id,tags
0,Avatar,19995,action adventure fantasy sciencefiction jakesu...
1,Pirates of the Caribbean: At World's End,285,adventure fantasy action captainjacksparrow wi...
2,Spectre,206647,action adventure crime jamesbond blofeld madel...
3,The Dark Knight Rises,49026,action crime drama thriller brucewayne batman ...
4,John Carter,49529,action adventure sciencefiction johncarter dej...
...,...,...,...
4804,El Mariachi,9367,action crime thriller elmariachi bigot n mauri...
4805,Newlyweds,72766,comedy romance buzzy linda marsha a newlywed c...
4806,"Signed, Sealed, Delivered",231617,comedy drama romance tvmovie olivero toole sha...
4807,Shanghai Calling,126186,sam amanda donald when ambitious new york atto...


In [31]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mbvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mbvin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [32]:
stop_words = set(stopwords.words('english'))

# Optional but STRONGLY recommended for movies
custom_stopwords = {
    "film", "movie", "movies", "story", "plot"
}

stop_words = stop_words.union(custom_stopwords)

In [33]:
lemmatizer = WordNetLemmatizer()

In [35]:
def clean_and_lemmatize(text):
    return " ".join(
        lemmatizer.lemmatize(w)
        for w in text.split()
        if w not in stop_words
    )

movies['tags'] = movies['tags'].apply(clean_and_lemmatize)

In [48]:
movies['tags']

0       action adventure fantasy sciencefiction jakesu...
1       adventure fantasy action captainjacksparrow wi...
2       action adventure crime jamesbond blofeld madel...
3       action crime drama thriller brucewayne batman ...
4       action adventure sciencefiction johncarter dej...
                              ...                        
4804    action crime thriller elmariachi bigot n mauri...
4805    comedy romance buzzy linda marsha newlywed cou...
4806    comedy drama romance tvmovie olivero toole sha...
4807    sam amanda donald ambitious new york attorney ...
4808    documentary ever since second grade first saw ...
Name: tags, Length: 4806, dtype: object

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(
    max_features=5000,      # critical: controls memory
    stop_words='english',   # extra safety
    ngram_range=(1, 2)      # captures phrases like "space travel"
)

In [59]:
vectors = cv.fit_transform(movies['tags']).toarray()

MemoryError: Unable to allocate 183. MiB for an array with shape (4806, 5000) and data type int64

In [39]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
similarity = cosine_similarity(vectors)

NameError: name 'vectors' is not defined

In [ ]:
def recommend(movie):
    movie_idx = movies[movies['title'] == movie].index[0]
    rank = sorted(list(enumerate(similarity[movie_idx])), reverse = True, key = lambda x: x[1])[1:6]
    frames = []
    for i in rank:
        frames.append(movies['title'].iloc[i])
    return frames

In [ ]:
similarity

array([[1.        , 0.11149893, 0.10984701, ..., 0.04441156, 0.02470831,
        0.        ],
       [0.11149893, 1.        , 0.05970814, ..., 0.04828045, 0.        ,
        0.        ],
       [0.10984701, 0.05970814, 1.        , ..., 0.04756515, 0.        ,
        0.        ],
       ...,
       [0.04441156, 0.04828045, 0.04756515, ..., 1.        , 0.06419407,
        0.04279605],
       [0.02470831, 0.        , 0.        , ..., 0.06419407, 1.        ,
        0.04761905],
       [0.        , 0.        , 0.        , ..., 0.04279605, 0.04761905,
        1.        ]], shape=(4806, 4806))

In [ ]:
recommend('Red Planet')

['The Martian',
 'Mission to Mars',
 'Apollo 18',
 'Sunshine',
 "The Astronaut's Wife"]

In [ ]:
recommend('Spectre')

['Never Say Never Again',
 'Quantum of Solace',
 'Skyfall',
 'Thunderball',
 'From Russia with Love']

In [ ]:
movies

,title,id,tags
0,Avatar,19995,action adventure fantasy sciencefiction jakesu...
1,Pirates of the Caribbean: At World's End,285,adventure fantasy action captainjacksparrow wi...
2,Spectre,206647,action adventure crime jamesbond blofeld madel...
3,The Dark Knight Rises,49026,action crime drama thriller brucewayne batman ...
4,John Carter,49529,action adventure sciencefiction johncarter dej...
...,...,...,...
4804,El Mariachi,9367,action crime thriller elmariachi bigot n mauri...
4805,Newlyweds,72766,comedy romance buzzy linda marsha newlywed cou...
4806,"Signed, Sealed, Delivered",231617,comedy drama romance tvmovie olivero toole sha...
4807,Shanghai Calling,126186,sam amanda donald ambitious new york attorney ...


In [ ]:
# import pickle
# pickle.dump(movies, open('movies.pkl', 'wb'))

In [ ]:
# pickle.dump(similarity, open('similarity.pkl', 'wb'))